In [1]:
#-*- coding : utf-8 -*-
# coding: utf-8

# 507170536 吳韋德 
# 507170524 張亞亭

import pandas as pd
from ipywidgets import *
import json
import glob
import os
from google_trans_new import google_translator
from IPython.display import display
from IPython.display import clear_output as clear


csv_df = None # 使用全域變數紀錄讀取的資料，方便其他 functon 使用
category_mapping = {} # 紀錄影片類別的字典 { id:類別 }
country_mapping = dict(美國='US', 日本='JP', 加拿大='CA', 德國='DE', 法國='FR',
                       英國='GB', 印度='IN', 墨西哥='MX', 南韓='KR', 全部='ALL')
country_mapping_revert = {v: k for k, v in country_mapping.items()} # 反轉 country_mapping，查詢使用
result = None # 資料分析的結果，提供給產出報表用

# 讀取紀錄影片類別的 json 檔案，並將結構轉換成字典 { id:類別 } 方便後續使用
def loadJson():
    path = 'data/US_category_id.json'
    global category_mapping
    with open(path, 'r') as obj:
        json_data = json.load(obj)
        for item in json_data['items']:
            id = int(item['id'])
            if(id not in category_mapping):
                category_mapping[id] = item['snippet']['title']

# 根據給定的參數資料，顯示前 n 筆資料，輸入參數為 (篩選的欄位, 要顯示幾筆, 排序:遞增或遞減 )
def show_top_n(properties, top_n, sort):
    csv_df[properties] = csv_df[properties].astype(int) # 資料型態轉成數字
    show_columns = ['title', properties]
    if len(csv_df.index) > 100000:
        show_columns.append('country')
    global result
    result = csv_df.sort_values(properties, ascending = sort)[show_columns]
    new_columns = []
    columns_mapping = {'title':'標題','views':'觀看次數','likes':'喜歡','dislikes':'不喜歡','comment_count':'留言數','country':'國家'}
    for column in result.columns:
        new_column = columns_mapping[column]
        new_columns.append(new_column)
    result.columns = new_columns
    display(result.head(top_n))

# 根據 video_id 欄位，移除重複的影片
def remove_duplicates():
    csv_df.drop_duplicates(subset='video_id', keep='last', inplace=True) # 有重複資料選擇最後一筆，並且直接替換舊的資料
    total = len(csv_df.index)
    return total


# 讀取影片資料(.csv)，參數為 ( 要讀取的國家 )
def loadData(country):
    global csv_df
    if country=='ALL':
        all_country_data =[]
        path = os.getcwd() + '\\data' # 設定資料的資料夾路徑
        path = os.path.abspath(path)  # 讀取指定資料夾內所有的 csv 路徑
        csv_paths = glob.glob(os.path.join(path, '*.csv')) # 讀取指定資料夾內所有的 csv 路徑組成 list
        print('已從 data 資料夾中讀取影片資料，資料處理中...\n')
        # 使用 for in 讀取每個 csv，在使用 concat 方法將所有 dataframe 接起來
        for f in csv_paths:
            print('\t'+f)
             # 找出是哪個國家的資料 ---- start
            strs = f.split('\\')
            fileName = strs[len(strs)-1]
            country_code = fileName.replace('videos.csv','')
            country_name = country_mapping_revert[country_code]
             # 找出是哪個國家的資料 ---- end
                
            temp_csv_df = pd.read_csv(f, encoding = 'utf-8', sep=',', error_bad_lines=False) # 讀取 csv
            
            # 在原本的 dataframe 新增國家的欄位 ---- start
            count = len(temp_csv_df.index)
            country_names = [country_name] * count
            temp_csv_df['country'] = country_names
            # 在原本的 dataframe 新增國家的欄位 ---- end
            
            all_country_data.append(temp_csv_df)
        csv_df = pd.concat(all_country_data) # 將每個國家的 dataframe 資料組合起來
        print('========================================================================================================\n')
        total = len(csv_df.index) # 原始資料量
        print('\n讀取' + country_mapping_revert[country] +'的影片資料，資料數量： ' + format(total,','))
        print('\n========================================================================================================\n')
        new_total = remove_duplicates() # 移除重複的資料
        print('移除重複的資料，新的資料數量：' + format(new_total,','))
        print('\n========================================================================================================\n')
        interact(show_top_n, properties=dict(觀看次數='views', 喜歡='likes', 不喜歡='dislikes', 留言數='comment_count'),
         top_n=(1,20), sort=dict(遞增=False, 遞減=True)) # 建立一個下拉選單，可以選擇屬性、資料筆數、排序方式，最後過濾出資料
    else:
        path = 'data/' + country + 'videos.csv'
        # 因為有多個語系，要設定 encoding = utf-8；原始資料某幾列的資料欄位數不對，會無法讀取，因此增加 error_bad_lines 忽略錯誤欄位
        csv_df = pd.read_csv(path, encoding = 'utf-8', sep=',', error_bad_lines=False) # 讀取 csv
        print('========================================================================================================\n')
        total = len(csv_df.index) # 原始資料量
        print('讀取' + country_mapping_revert[country] +'的影片資料，資料數量： ' + format(total,','))
        print('\n========================================================================================================\n')
        new_total = remove_duplicates() # 移除重複的資料
        
        print('移除重複的資料，新的資料數量：' + format(new_total,','))
        print('\n========================================================================================================\n')
        # 集中程度計算：(原始資料量 - 去重複後資料量) / 原始資料量 * 100
        concentrated = round(((total - new_total) / total) * 100, 2)
        print('熱門影片集中程度：' + str(concentrated) + '%')
        print('\n========================================================================================================\n')
        interact(show_top_n, properties=dict(觀看次數='views', 喜歡='likes', 不喜歡='dislikes', 留言數='comment_count'),
         top_n=(1,20), sort=dict(遞增=False, 遞減=True)) # 建立一個下拉選單，可以選擇屬性、資料筆數、排序方式，最後過濾出資料

loadJson() # 讀取存放影片類別的檔案(.json)

interact(loadData, country=country_mapping) # 建立一個下拉選單，可以選擇國家，並讀取該國家的影片資料

interactive(children=(Dropdown(description='country', options={'美國': 'US', '日本': 'JP', '加拿大': 'CA', '德國': 'DE'…

<function __main__.loadData(country)>

In [2]:
categorys = {} # 每個類別的數量紀錄

# 根據類別 id 找出對應類別
def findCategoryById(id):
    global categorys
    global category_mapping
    title = category_mapping[id]
    if title in categorys: # 如果類別已經在字典裡，數量加 1
        categorys[title] = categorys[title]+1
    else: # 如果類別沒有在字典裡，新增類別
        categorys[title] = 1

# 根據國別統計熱門影片的類別偏好
def findCategorys(country):
    global categorys
    categorys = {}
    path = 'data/' + country + 'videos.csv'
    data_df = pd.read_csv(path, encoding = 'utf-8', sep=',', error_bad_lines=False) # 讀取 csv 資料
    data_df2 = data_df.drop_duplicates(subset='video_id', keep='last') # 去除重複資料
    for index, row in data_df2.iterrows():
        findCategoryById(row['category_id']) # 每一筆資料使用 findCategoryById 函式找出對應的類別
    print('統計影片數量：' + str(len(data_df2.index)))
    sorted_result = sorted(categorys.items(), key=lambda x:x[1], reverse=True) # 使用 lambda 函式，根據字典的 value 排序
    sorted_result_list = list(sorted_result)
    
    translator = google_translator() # 啟動 google 翻譯
    translate_data = []
    for cate in sorted_result_list:
        translate_text = translator.translate(cate[0], lang_src='en', lang_tgt='zh-tw') # 翻譯英文成繁體中文
        translate_data.append(translate_text)
    global result
    result = pd.DataFrame(sorted_result_list, columns=['影片類別', '數量']) # 轉換成 dataframe
    result['類別中文'] = translate_data # 新增 dataframe 欄位
    display(result) 

interact(findCategorys, country=dict(美國="US", 日本="JP", 加拿大='CA', 德國='DE', 法國='FR',
                                英國='GB', 印度='IN', 墨西哥='MX', 南韓='KR'))


interactive(children=(Dropdown(description='country', options={'美國': 'US', '日本': 'JP', '加拿大': 'CA', '德國': 'DE'…

<function __main__.findCategorys(country)>

In [3]:
authors = {} # 每個作者的影片數量
data_df2 = None

# 根據輸入值 n 可以決定要顯示的資料筆數
def show_result(top_n):
    global data_df2 # 使用全域變數
    total = len(data_df2.index)
    print('統計數量：' + str(total))
    sorted_result = sorted(authors.items(), key=lambda x:x[1], reverse=True) # 使用 lambda 函式，根據字典的 value 排序
    sorted_result_list = list(sorted_result)
    global result
    result = pd.DataFrame(sorted_result_list, columns=['作者', '數量'])
    display(result.head(top_n)) # 產生 dataframe 並顯示

# 根據國別統計熱門影片的作者排名
def findAuthors(country):
    global authors # 使用全域變數
    global data_df2 # 使用全域變數
    path = 'data/' + country + 'videos.csv'
    data_df = pd.read_csv(path, encoding = 'utf-8', sep=',', error_bad_lines=False)  # 讀取 csv 資料
    data_df2 = data_df.drop_duplicates(subset='video_id', keep='last') # 去除重複資料
    authors = {}
    for index, row in data_df2.iterrows(): # 迭代每一筆 dataframe
        author = row['channel_title']
        if author in authors: # 如果作者已經在字典裡，數量加 1
            authors[author] = authors[author]+1
        else:
            authors[author] = 1 # 如果作者沒有在字典裡，新增作者
    interact(show_result, top_n=(1,20)) # 提供數量選單，讓使用者決定顯示資料筆數
    
    
interact(findAuthors, country=dict(美國="US", 日本="JP", 加拿大='CA', 德國='DE', 法國='FR',
                                英國='GB', 印度='IN', 墨西哥='MX', 南韓='KR'))

interactive(children=(Dropdown(description='country', options={'美國': 'US', '日本': 'JP', '加拿大': 'CA', '德國': 'DE'…

<function __main__.findAuthors(country)>

In [4]:

# 找出綜合評分最高的影片
def comprehensive(country):
    
    path = 'data/' + country + 'videos.csv'
    # 因為有多個語系，要設定 encoding = utf-8；原始資料某幾列的資料欄位數不對，會無法讀取，因此增加 error_bad_lines 忽略錯誤欄位
    data_df = pd.read_csv(path, encoding = 'utf-8', sep=',', error_bad_lines=False) # 讀取 csv
    new_data_df = data_df.drop_duplicates(subset='video_id', keep='last') # 有重複資料選擇最後一筆
    
    total_views = new_data_df['views'].sum() # 這個 dataframe 的總觀看數
    total_like = new_data_df['likes'].sum() # 這個 dataframe 的總喜歡數
    total_dislike = new_data_df['dislikes'].sum() # 這個 dataframe 的總不喜歡數
    total_comment = new_data_df['comment_count'].sum() # 這個 dataframe 的總評論數
    
    data_size = len(new_data_df.index)
    
    print('平均觀看次數：'+ format(round(total_views / data_size),','))
    print('平均喜歡人數：'+ format(round(total_like / data_size),','))
    print('平均不喜歡人數：'+ format(round(total_dislike / data_size),','))
    print('平均評論數：'+ format(round(total_comment / data_size),','))
    
    likes = total_like - total_dislike
    
    view_point = likes / total_views # 計算每個觀看次數值多少分
    like_point = 1 # 使用喜歡的數量當作基準值 1
    comment_point = likes / total_comment # 計算每個評論值多少分
    
    new_data_df['point'] = 0 # 初始化欄位 point
    
    # 計算每筆資料的分數
    def cal_point(row):
        point = 0
        point += (row['likes'] - row['dislikes']) * like_point # 計算喜歡分數
        point += row['views'] * view_point # 計算觀看分數
        point += row['comment_count'] * comment_point # 計算評論數分數
        row['point'] = point / 10000
        return row
    new_data_df = new_data_df.apply(cal_point, axis=1) # 針對每個欄位執行給定的方法
    global result
    result = new_data_df.sort_values('point', ascending = False)[['title','point','views','likes','dislikes','comment_count']]
    display(result.head(10))
    
interact(comprehensive, country=dict(美國="US", 日本="JP", 加拿大='CA', 德國='DE', 法國='FR',
                                英國='GB', 印度='IN', 墨西哥='MX', 南韓='KR'))


interactive(children=(Dropdown(description='country', options={'美國': 'US', '日本': 'JP', '加拿大': 'CA', '德國': 'DE'…

<function __main__.comprehensive(country)>

In [5]:
global result
report_type = 'Excel' # 導出的報表類別，預設是 Excel

# 切換選單時執行這個方法，修改 report_type
def choice_type(change):
    global report_type
    report_type = change.new

# 建立下拉選單並監聽 change 事件
option_list = ('Excel', 'Csv', 'Json')
dropdown = Dropdown(description="報表類型：", options=option_list)
dropdown.observe(choice_type, names='value')
display(dropdown)

# 點導出報表按鈕時執行這個方法，根據 report_type 產生對應的格式
def on_export(b):
    clear()
    display(dropdown)
    display(HBox([exportBtn]))
    display(HBox([previewBtn]))
    global report_type
    if report_type == 'Excel':
        print('導出 Excel 報表')
        result.to_excel('report.xls', sheet_name='sheet1')
    elif report_type == 'Csv':
        print('導出 Csv 報表')
        result.to_csv('report.csv')
    else:
        print('導出 Json 報表')
        with open('report.json', 'w', encoding='utf-8') as file:
            result.to_json(file, orient='index', force_ascii=False)
        
# 建立導出報表按鈕，並監聽 click 事件
exportBtn = Button(description="導出報表")
exportBtn.on_click(on_export)
display(HBox([exportBtn]))

# 點預覽資料按鈕時執行這個方法，顯示 result 的資料內容
def on_preview(b):
    clear()
    display(dropdown)
    display(HBox([exportBtn]))
    display(HBox([previewBtn]))
    display(result)

# 建立導出報表按鈕，並監聽 click 事件
previewBtn = Button(description="預覽資料")
previewBtn.on_click(on_preview)
display(HBox([previewBtn]))

Dropdown(description='報表類型：', options=('Excel', 'Csv', 'Json'), value='Excel')

In [6]:
import warnings
warnings.filterwarnings('ignore') # 隱藏錯誤訊息

global result

indexes = ['title', '標題', '影片類別', '作者'] # 圖的欄位
numbers = ['觀看次數','喜歡','不喜歡','留言數','point', '數量'] # 圖的資料
index = 0
df_to_draw = None

# 使用 while 加上例外處理，尋找要繪製的資料內，欄位的正確名字
while True:
    try:
        df_to_draw = result.set_index(indexes[index])
        break
    except KeyError:
        index += 1
    except:
        print('something wrong')
        break

df_to_draw = df_to_draw.head(10) # 取前 10 筆資料繪圖

# 根據不同的圖表類別，繪製圖表
def draw(kind):
    n_index = 0
    # 使用 while 加上例外處理，尋找要繪製的資料內，資料的正確名字
    while True:
        try:
            df_to_draw[numbers[n_index]].plot(kind=kind)
            break
        except KeyError:
            n_index += 1
        except:
            print('something wrong')
            break

interact(draw, kind=dict(柱狀圖="bar", 橫向柱狀圖="barh", 派餅圖="pie"))

interactive(children=(Dropdown(description='kind', options={'柱狀圖': 'bar', '橫向柱狀圖': 'barh', '派餅圖': 'pie'}, valu…

<function __main__.draw(kind)>